In [4]:
import pandas as pd
from datetime import date, datetime
from datetime import datetime as dt

In [5]:
concated2020 = pd.read_csv("concated_mainyear_2020.csv").reset_index(drop=True)
concated2020 = concated2020.iloc[:, 1:] 
concated2021 = pd.read_csv("concated_mainyear_2021.csv").reset_index(drop=True)
concated2021 = concated2021.iloc[:, 1:] 
concated2022 = pd.read_csv("concated_mainyear_2022.csv").reset_index(drop=True)
concated2022 = concated2022.iloc[:, 1:] 
concated2023 = pd.read_csv("concated_mainyear_2023.csv").reset_index(drop=True)
concated2023 = concated2023.iloc[:, 1:] 
df = pd.concat([concated2020,
               concated2021,
               concated2022,
               concated2023],
              ignore_index=True)

In [6]:
newdf = df.copy()
newdf['Tarih'] = pd.to_datetime(df['Tarih'], format="%d.%m.%Y")

newdf['Fon Kodu'] = df['Fon Kodu'].astype(str)
newdf['Fon Adı'] = df['Fon Adı'].astype(str)

newdf['Fiyat'] = pd.to_numeric(df['Fiyat'].astype(str).str.replace(",", ""))
#newdf['Fiyat'] = newdf.apply(lambda x: "{:,}".format(x['Fiyat']), axis=1)

newdf['Tedavüldeki Pay Sayısı'] = pd.to_numeric(df['Tedavüldeki Pay Sayısı'].astype(str).str.replace(",", ""))
#newdf['Tedavüldeki Pay Sayısı'] = newdf.apply(lambda x: "{:,}".format(x['Tedavüldeki Pay Sayısı']), axis=1)

newdf['Fon Toplam Değer'] = pd.to_numeric(df['Fon Toplam Değer'].astype(str).str.replace(",", ""))
#newdf['Fon Toplam Değer'] = newdf.apply(lambda x: "{:,}".format(x['Fon Toplam Değer']), axis=1)

newdf['Kişi Sayısı'] = pd.to_numeric(df['Kişi Sayısı'].astype(str).str.replace(",", ""))
#newdf['Kişi Sayısı'] = newdf.apply(lambda x: "{:,}".format(x['Kişi Sayısı']), axis=1)

newdf = newdf.sort_values(by=['Fon Kodu', 'Tarih'])
newdf['Günlük Getiri'] = newdf.groupby('Fon Kodu')['Fiyat'].pct_change()
# newdf['Günlük Getiri2'] = (newdf['Fiyat'] / newdf.groupby('Fon Kodu')['Fiyat'].shift(1)) - 1 
## pct change komutu aynı işlevi yaptığı için iptal ettim.
newdf['Günlük Para Hareketi'] = newdf['Fon Toplam Değer'] - ((1 + newdf['Günlük Getiri']) * newdf.groupby('Fon Kodu')['Fon Toplam Değer'].shift(1))
newdf = newdf.sort_values(by=["Tarih","Fon Kodu"]).reset_index(drop=True)


In [3]:

kurlar = pd.read_csv("kurlar.csv")
kurlar = kurlar.iloc[:, 1:] 
kurlar["Tarih"] = pd.to_datetime(kurlar["Tarih"])
kurlar = kurlar.loc[kurlar["Kod"]=="USD"]

newdf = pd.merge(newdf, kurlar[["Tarih","ForexBuying"]], on='Tarih', how='left').rename(columns={"ForexBuying":"ForexBuyingUSD"})
newdf['FiyatUSD'] = newdf['Fiyat'] / newdf['ForexBuyingUSD']

# https://tr.investing.com/currencies/gau-try-historical-data
altin = pd.read_csv("altin.csv")
altin["Tarih"] = pd.to_datetime(altin["Tarih"])
newdf = pd.merge(newdf, altin[["Tarih","Gram Altın Alış Fiyatı"]], on='Tarih', how='left').rename(columns={"Gram Altın Alış Fiyatı":"Gram Altın Fiyatı TL"})

# https://tr.investing.com/currencies/xagg-try-historical-data
gumus = pd.read_csv("gumus.csv")
gumus["Tarih"] = pd.to_datetime(gumus["Tarih"],dayfirst=True)
gumus["Düşük"] = pd.to_numeric(gumus["Düşük"].str.replace(",","."))
gumus = gumus[["Tarih","Düşük"]].rename(columns={"Düşük":"Gram Gümüş Alış Fiyatı"})
newdf = pd.merge(newdf, gumus[["Tarih","Gram Gümüş Alış Fiyatı"]], on='Tarih', how='left').rename(columns={"Gram Gümüş Alış Fiyatı":"Gram Gümüş Fiyatı TL"})

# https://tr.investing.com/indices/ise-100-historical-data
bist100 = pd.read_csv("bist100.csv")
bist100["Tarih"] = pd.to_datetime(bist100["Tarih"],dayfirst=True)
bist100["Düşük"] = pd.to_numeric(bist100["Düşük"].str.replace(".","").str.replace(",","."))
bist100 = bist100[["Tarih","Düşük"]].rename(columns={"Düşük":"Bist 100 Fiyatı"})
newdf = pd.merge(newdf, bist100[["Tarih","Bist 100 Fiyatı"]], on='Tarih', how='left').rename(columns={"Bist 100 Fiyatı":"Bist 100 Fiyatı TL"})

# https://tr.investing.com/indices/ise-30-historical-data
bist30 = pd.read_csv("bist30.csv")
bist30["Tarih"] = pd.to_datetime(bist30["Tarih"],dayfirst=True)
bist30["Düşük"] = pd.to_numeric(bist30["Düşük"].str.replace(".","").str.replace(",","."))
bist30 = bist30[["Tarih","Düşük"]].rename(columns={"Düşük":"Bist 30 Fiyatı"})
newdf = pd.merge(newdf, bist30[["Tarih","Bist 30 Fiyatı"]], on='Tarih', how='left').rename(columns={"Bist 30 Fiyatı":"Bist 30 Fiyatı TL"})




ValueError: time data "13.01.2020" doesn't match format "%m.%d.%Y", at position 10. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
kurlar = kurlar.rename(columns={"ForexBuying": "Fiyat", "Kod": "Fon Kodu", "Isim": "Fon Adı"})
kurlar["Fon Kodu"] = kurlar["Fon Kodu"].str.replace("USD", "USDX")

altin = altin.rename(columns={"Gram Altın Alış Fiyatı": "Fiyat"})
altin["Fon Kodu"] = "ALTINX"
altin["Fon Adı"] = "Gram Altın Fiyatı TL"

gumus = gumus.rename(columns={"Gram Gümüş Alış Fiyatı": "Fiyat"})
gumus["Fon Kodu"] = "GUMUSX"
gumus["Fon Adı"] = "Gram Gümüş Fiyatı TL"

bist100 = bist100.rename(columns={"Bist 100 Fiyatı": "Fiyat"})
bist100["Fon Kodu"] = "BIST100X"
bist100["Fon Adı"] = "BIST100 Fiyatı TL"

bist30 = bist30.rename(columns={"Bist 30 Fiyatı": "Fiyat"})
bist30["Fon Kodu"] = "BIST30X"
bist30["Fon Adı"] = "BIST30 Fiyatı TL"



newdf = pd.concat([newdf,kurlar,altin,gumus,bist100,bist30],axis=0)


In [ ]:
newdf.to_csv("newdf2.csv", index=False, sep='\t', encoding="utf-8", decimal=',')

In [7]:
newdf.loc[newdf["Fon Kodu"]=="AAV"]

,Tarih,Fon Kodu,Fon Adı,Fiyat,Tedavüldeki Pay Sayısı,Kişi Sayısı,Fon Toplam Değer,Günlük Getiri,Günlük Para Hareketi
3,2020-01-02,AAV,ATA PORTFÖY İKİNCİ HİSSE SENEDİ (TL) FONU (HİS...,0.031263,2.868807e+09,11,8.968858e+07,NaN,NaN
433,2020-01-03,AAV,ATA PORTFÖY İKİNCİ HİSSE SENEDİ (TL) FONU (HİS...,0.031650,2.868807e+09,11,9.079710e+07,0.012379,-1.725928e+03
863,2020-01-06,AAV,ATA PORTFÖY İKİNCİ HİSSE SENEDİ (TL) FONU (HİS...,0.030836,3.146594e+09,11,9.702699e+07,-0.025719,8.565086e+06
1293,2020-01-07,AAV,ATA PORTFÖY İKİNCİ HİSSE SENEDİ (TL) FONU (HİS...,0.030065,3.146594e+09,11,9.460224e+07,-0.025003,1.238389e+03
1723,2020-01-08,AAV,ATA PORTFÖY İKİNCİ HİSSE SENEDİ (TL) FONU (HİS...,0.030516,3.146594e+09,11,9.602002e+07,0.015001,-1.331089e+03
...,...,...,...,...,...,...,...,...,...
848763,2023-12-25,AAV,ATA PORTFÖY İKİNCİ HİSSE SENEDİ (TL) FONU (HİS...,32.471813,2.646394e+07,2031,8.593322e+08,-0.017532,-2.270380e+05
850195,2023-12-26,AAV,ATA PORTFÖY İKİNCİ HİSSE SENEDİ (TL) FONU (HİS...,31.322040,2.629390e+07,2032,8.235785e+08,-0.035408,-5.326203e+06
851627,2023-12-27,AAV,ATA PORTFÖY İKİNCİ HİSSE SENEDİ (TL) FONU (HİS...,31.505909,2.630338e+07,2022,8.287118e+08,0.005870,2.987033e+05
853059,2023-12-28,AAV,ATA PORTFÖY İKİNCİ HİSSE SENEDİ (TL) FONU (HİS...,31.327269,2.642329e+07,2020,8.277696e+08,-0.005670,3.756560e+06
